# Chromvar rescore topics

In [5]:
%load_ext autoreload
%autoreload 2
    
import os
os.environ["RAY_TQDM_PATCH_PRINT"]="0"
from umap import *
import numpy as np
import scprinter as scp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
import pandas as pd
import numpy as np
import os
import pickle
import torch
import matplotlib as mpl
import scanpy as sc
import anndata as ad
from scmallet import *
mpl.rcParams['pdf.fonttype'] = 42

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import scanpy as sc
import anndata
import cupy as cp
import cupyx as cpx
import time

import warnings
warnings.filterwarnings("ignore")
import rmm
from rmm.allocators.cupy import rmm_cupy_allocator
rmm.reinitialize(
    managed_memory=True, # Allows oversubscription
    pool_allocator=True, # default is False
    devices=0, # GPU device IDs to register. By default registers only GPU 0.
)

cp.cuda.set_allocator(rmm_cupy_allocator)

In [6]:
work_dir = '/data/rzhang/data_15/colitisDSS/tissue'
meta = 'DSS-prim-merge10-ATAC.qc_stats.singlet_epi_only.txt'
frags = 'DSS-prim-merge10-ATAC.reshift.fragments.tsv.gz'
peak = 'DSS-prim-merge10-ATAC.cleanPeaks.fixedwidthpeaks_800bp_300reSized.bed'

In [7]:
atac_adata = ad.read_h5ad(f'{work_dir}/cellxpeak.h5ad')

In [8]:
rna_adata = sc.read_mtx('/data/rzhang/data_15/colitisDSS/tissue/joint_topic/DSS-prim-merge10-RNA.raw_UMI.merged.converted_barcodes.mtx.gz')
rna_adata = ad.AnnData(X = rna_adata.X.T)
bc = [line.strip() for line in open('/data/rzhang/data_15/colitisDSS/tissue/joint_topic/DSS-prim-merge10-RNA.raw_UMI.merged.converted_barcodes.barcodes.list', 'r').readlines()]
rna_adata.obs.index = bc
bc = [line.strip() for line in open('/data/rzhang/data_15/colitisDSS/tissue/joint_topic/DSS-prim-merge10-RNA.raw_UMI.merged.converted_barcodes.genes.list', 'r').readlines()]
rna_adata.var.index = bc

In [9]:
meta_cell = pd.read_csv('/data/rzhang/data_15/colitisDSS/tissue/joint_topic/DSS-prim-merge10-ATAC.qc_stats.singlet.txt', sep='\t')shared = list(set(meta_cell['barcode']) &  set(atac_adata.obs.index)  & set(rna_adata.obs.index))
atac_adata = atac_adata[shared].copy()
rna_adata = rna_adata[shared].copy()

In [11]:
atac_feats = pd.read_csv("/data/rzhang/data_15/colitisDSS/tissue/joint_chromvar/DSS-prim-merge10.ATAC.top_features.1000.annot.txt.gz", sep='\t')
atac_bg = pd.read_csv("/data/rzhang/data_15/colitisDSS/tissue/joint_chromvar/DSS-prim-merge10.atac_bg.csv.gz", header=None)

In [20]:
atac_feat_weighted = pd.read_csv("/data/rzhang/data_15/colitisDSS/tissue/joint_topic/topic90_region_topics_ATAC.txt.gz", sep='\t', index_col=0)

In [15]:
atac_adata.varm['bg_peaks'] = atac_bg.values.astype('int')  - 1  # minus 1 because R starts at 1, python starts at 0
atac_adata.varm['motif_match'] = atac_feats.values
atac_adata.uns['motif_name'] = atac_feats.columns

In [16]:
chromvar_atac_surya = scp.chromvar.compute_deviations(atac_adata, chunk_size=50000, device='cuda')

Computing expectation reads per cell and peak...


Processing chunks:   0%|          | 0/2 [00:00<?, ?it/s]

Processing background peaks:   0%|          | 0/250 [00:00<?, ?it/s]

Processing background peaks:   0%|          | 0/250 [00:00<?, ?it/s]

In [21]:
atac_adata.varm['motif_match'] = atac_feat_weighted.values
chromvar_atac_raw_weighted = scp.chromvar.compute_deviations(atac_adata, chunk_size=50000, device='cuda')

Computing expectation reads per cell and peak...


Processing chunks:   0%|          | 0/2 [00:00<?, ?it/s]

Processing background peaks:   0%|          | 0/250 [00:00<?, ?it/s]

Processing background peaks:   0%|          | 0/250 [00:00<?, ?it/s]

In [1]:
# Now for RNA

In [32]:
rna_feats = pd.read_csv("/data/rzhang/data_15/colitisDSS/tissue/joint_chromvar/DSS-prim-merge10.RNA.top_features.100.annot.txt.gz", sep='\t')
rna_bg = pd.read_csv("/data/rzhang/data_15/colitisDSS/tissue/joint_chromvar/DSS-prim-merge10.rna_bg.csv.gz", header=None)

In [33]:
rna_feat_weighted = pd.read_csv("/data/rzhang/data_15/colitisDSS/tissue/joint_topic/topic90_region_topics_RNA.txt.gz", sep='\t', index_col=0)

In [35]:
rna_adata = rna_adata[:, rna_feat_weighted.index].copy()

In [36]:
rna_adata.varm['bg_peaks'] = rna_bg.values.astype('int')  - 1
rna_adata.varm['motif_match'] = rna_feat_weighted.loc[rna_adata.var.index].values
rna_adata.uns['motif_name'] = rna_feat_weighted.columns

In [37]:
rna_adata.X = rna_adata.X.tocsr()

In [38]:
chromvar_rna = scp.chromvar.compute_deviations(rna_adata, chunk_size=50000, device='cuda')

Computing expectation reads per cell and peak...


Processing chunks:   0%|          | 0/2 [00:00<?, ?it/s]

Processing background peaks:   0%|          | 0/250 [00:00<?, ?it/s]

Processing background peaks:   0%|          | 0/250 [00:00<?, ?it/s]

In [46]:
chromvar_atac_df = pd.DataFrame(chromvar_atac_raw_weighted.X, index=chromvar_atac.obs.index, columns=chromvar_atac.var.index)
chromvar_rna_df = pd.DataFrame(chromvar_rna.X, index=chromvar_rna.obs.index, columns=chromvar_rna.var.index)

In [48]:
chromvar_atac_df

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,...,topic80,topic81,topic82,topic83,topic84,topic85,topic86,topic87,topic88,topic89
"R1.031,R2.046,R3.075,P1.70",-1.366641,0.143261,-1.270097,-0.371674,-0.088459,-1.688468,-0.752315,-0.429030,-0.415447,-0.355882,...,-0.037278,0.459639,0.446594,1.323321,4.577820,-2.675338,0.568658,-2.156944,-2.638350,2.462148
"R1.050,R2.071,R3.048,P1.64",-1.507383,1.412635,-0.577062,1.757667,1.087432,-1.179368,-2.706171,0.206149,-1.550619,0.328392,...,-3.631345,-0.910744,0.356931,-2.051718,-3.826802,-1.998591,-0.619211,-0.820506,-1.119303,-2.736650
"R1.056,R2.080,R3.007,P1.10",-0.031410,-0.290445,-2.190396,-1.162622,-1.282947,0.804657,1.953583,-1.061617,-0.331798,-0.747013,...,-4.465086,1.461688,0.969166,-1.913750,-0.509255,0.364759,-0.017512,-1.310513,2.692523,-1.734738
"R1.075,R2.074,R3.034,P1.03",0.388055,-0.658049,-0.938044,1.817200,-0.814041,5.112604,0.577426,0.136579,-2.184929,-0.539473,...,-3.209621,0.203013,-2.161755,-1.553014,-2.619650,0.784368,-1.636377,-0.870906,1.337391,-1.706807
"R1.022,R2.059,R3.078,P1.70",-1.556754,-0.118766,0.063000,-0.215137,1.020465,1.072984,0.103899,-1.160621,-0.269324,-1.688045,...,-0.431222,-0.070306,-2.217784,1.454190,3.599430,-1.653373,-0.057552,1.056109,-0.564561,0.151325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"R1.086,R2.079,R3.006,P1.58",1.006248,1.268830,0.803794,0.113502,2.100283,-0.267891,-0.683890,-2.063905,2.075983,0.577804,...,1.057477,1.390601,-0.152740,-0.063387,-1.739297,2.650687,0.003606,-0.473538,1.050352,-1.166662
"R1.008,R2.085,R3.071,P1.58",1.935584,1.654158,-0.834974,-0.386435,-0.981689,2.934273,1.339843,-0.271877,1.198048,1.315380,...,2.154189,-1.286137,-0.361372,0.809753,-1.678544,1.273655,1.759383,-0.439327,-1.003670,-1.492882
"R1.060,R2.054,R3.013,P1.49",-1.312132,-0.973294,2.341133,-1.310314,0.435399,1.344081,-1.119868,-1.452120,0.290307,-0.300064,...,0.728945,-1.171426,1.885644,1.027861,0.926919,-2.222029,-0.149075,-0.793204,0.410778,2.518695
"R1.024,R2.032,R3.080,P1.58",0.793664,-0.199103,0.499222,-1.273961,-0.123426,-1.168824,0.854876,-0.217302,2.023154,1.460193,...,-2.197592,-0.370847,-0.256638,0.358157,-4.190166,1.848474,1.772627,0.431719,-0.922218,-0.856660


In [49]:
chromvar_rna_df

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,...,topic80,topic81,topic82,topic83,topic84,topic85,topic86,topic87,topic88,topic89
"R1.031,R2.046,R3.075,P1.70",2.130437,-0.238110,1.878019,-0.835568,-0.199869,-0.351800,4.780596,0.110598,-0.705272,3.819647,...,-0.396872,-0.482384,0.116347,-1.660009,-0.255327,0.371710,0.107414,-0.299008,-0.468851,-0.202749
"R1.050,R2.071,R3.048,P1.64",22.417719,0.020989,15.857437,-0.251592,0.055208,-0.003256,4.803228,-0.699828,-0.178209,0.288058,...,0.381816,-0.069678,-0.628139,-0.490852,0.039316,-0.144884,-0.526118,0.001613,-0.035150,0.047567
"R1.056,R2.080,R3.007,P1.10",-3.338222,0.403929,-2.344495,0.433058,0.471633,0.458254,-0.712568,2.606498,0.224673,-2.910448,...,0.255550,0.479721,2.483101,0.076412,0.413672,1.507378,2.148255,0.373528,0.407293,0.416165
"R1.075,R2.074,R3.034,P1.03",-1.023161,-0.147414,-0.158328,-0.545283,-0.147076,-0.208330,-1.345599,-0.599200,-0.414735,2.803222,...,-0.484143,-0.283394,-0.544202,-0.989034,-0.164187,0.118560,-0.437664,-0.177218,-0.257092,-0.156130
"R1.022,R2.059,R3.078,P1.70",-1.142916,0.487472,-0.709428,0.911307,0.497963,0.628820,1.639783,1.022455,0.772583,-2.592395,...,0.239774,0.585358,0.941071,1.231855,0.498566,1.153308,0.838195,0.515513,0.563010,0.489692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"R1.086,R2.079,R3.006,P1.58",-1.289366,2.310359,-0.696279,2.652214,2.303403,2.635063,-1.109661,-0.790805,2.821274,-1.758425,...,0.888408,2.733129,-0.703154,2.794600,2.297762,1.230880,-0.594969,2.484560,2.657429,2.251925
"R1.008,R2.085,R3.071,P1.58",-1.651892,2.013955,-1.070968,2.621236,2.026769,2.331855,-1.432126,-0.338875,2.674708,-2.280592,...,1.642261,2.492877,-0.265487,2.749225,2.032020,1.386040,-0.130023,2.121918,2.490828,1.931020
"R1.060,R2.054,R3.013,P1.49",-2.084533,0.071484,-1.552979,-0.141948,0.107679,-0.001063,-0.830199,3.419373,-0.485020,-2.477851,...,0.495680,0.047813,3.174365,-0.705855,0.048024,0.554774,2.855780,-0.069815,-0.027257,0.111294
"R1.024,R2.032,R3.080,P1.58",-2.232380,1.577995,-1.864879,1.676914,1.731563,1.779182,-1.996069,-0.675782,2.047060,-2.716656,...,3.211079,1.635055,-0.612200,2.186357,1.540788,1.043859,-0.466168,1.604197,1.732836,1.563590


In [47]:
chromvar_atac_df.to_csv('/data/rzhang/data_15/colitisDSS/tissue/joint_chromvar/chromvar_atac.sep.weighted.txt.gz', sep='\t')
chromvar_rna_df.to_csv('/data/rzhang/data_15/colitisDSS/tissue/joint_chromvar/chromvar_rna.sep.weighted.txt.gz', sep='\t')